<pre>
1. Download all the data in this folder https://drive.google.com/open?id=1Z4TyI7FcFVEx8qdl4jO9qxvxaqLSqoEu. it contains two file both images and labels. The label file list the images and their categories in the following format:
            <b>path/to/the/image.tif,category</b>
            
    where the categories are numbered 0 to 15, in the following order:

    <b>0 letter
    1 form
    2 email
    3 handwritten
    4 advertisement
    5 scientific report
    6 scientific publication
    7 specification
    8 file folder
    9 news article
    10 budget
    11 invoice
    12 presentation
    13 questionnaire
    14 resume
    15 memo</b>
    
2. On this image data, you have to train 3 types of models as given below. You have to split the data into Train and Validation data.

3. Try not to load all the images into memory, use the gernarators that we have given the reference notebooks to load the batch of images only during the train data.
or you can use this method also
<a href='https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1'>https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1</a>

<a href='https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c'>https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c</a>


4. You are free to choose Learning rate, optimizer, loss function, image augmentation, any hyperparameters. but you have to use the same architechture what we are asking below. 

5. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

Note: fit_genarator() method will have problems with the tensorboard histograms, try to debug it, if you could not do use histgrams=0 i.e don't include histograms, check the documentation of tensorboard for more information. 

6. You can check about Transfer Learning in this link - <a href='https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html'>https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html</a>
</pre>

### Model-3

<pre>
1. Use same network as Model-2 '<b>INPUT --> VGG-16 without Top layers(FC) --> 2 Conv Layers identical to FC --> Output Layer</b>' and train only Last 6 Layers of VGG-16 network, 2 Conv layers identical to FC layers, 1 output layer.
</pre>

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive3 = GoogleDrive(gauth)

In [0]:
link='https://drive.google.com/open?id=1Z4TyI7FcFVEx8qdl4jO9qxvxaqLSqoEu'
fluff, id = link.split('=')
downloaded = drive3.CreateFile({'id':id}) 
downloaded.GetContentFile('rvl-cdip.rar') 


In [0]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
#importing tensorflow
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten
from tensorflow.keras.models import Model
import random as rn

import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt



Using TensorFlow backend.


In [0]:
pip install rarfile

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [0]:
import os

# define the name of the directory to be created
path = "/content/drive/My Drive/Colab Notebooks/TL/rvl"

try:
    os.makedirs(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory /content/drive/My Drive/Colab Notebooks/TL/rvl 


In [0]:
import rarfile
def unrar(dpath,xpath):
  for rar in os.listdir(dpath):
    filepath = os.path.join(dpath, rar)
    with rarfile.RarFile(filepath) as opened_rar:
      for f in opened_rar.infolist():
        print (f.filename, f.file_size)
        opened_rar.extractall(xpath)

unrar("/content/drive/My Drive/Colab Notebooks/TL/","/content/drive/My Drive/Colab Notebooks/TL/rvl")

In [0]:
dir_path = "/content/drive/My Drive/Colab Notebooks/TL/rvl"

In [0]:
os.listdir(dir_path)

['data_final', 'labels_final.csv']

In [0]:
import pandas
from keras_preprocessing.image import ImageDataGenerator
data = pandas.read_csv('/content/drive/My Drive/Colab Notebooks/TL/rvl/labels_final.csv') 
#y = data['label'].values
#X = data.drop(['label'], axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data,test_size=0.33)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data, test_size=0.33)
X_train, X_cv = train_test_split(X_train,test_size=0.33)

In [0]:
def append_ext(fn):
    return str(fn)



def append_ext_path(fn):
    return "/content/drive/My Drive/Colab Notebooks/TL/rvl/data_final/"+fn

X_train["path"]=X_train["path"].apply(append_ext_path)
X_train["path"]=X_train["path"].apply(append_ext)
X_train["label"]=X_train["label"].apply(append_ext)

X_test["label"]=X_test["label"].apply(append_ext)
X_test["path"]=X_test["path"].apply(append_ext_path)
X_test["path"]=X_test["path"].apply(append_ext)

In [0]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [0]:
X_train["path"]

In [0]:
from keras import backend as K
K.clear_session()

In [0]:

 
train_generator=datagen.flow_from_dataframe(
dataframe=X_train,
directory=None,
x_col="path",
y_col="label",
subset="training",
batch_size=32,
seed=10,
shuffle=True,
class_mode="categorical",
target_size=(156,256))

Found 16161 validated image filenames belonging to 16 classes.


In [0]:

 
valid_generator=datagen.flow_from_dataframe(
dataframe=X_train,
directory=None,
x_col="path",
y_col="label",
subset="validation",
batch_size=32,
seed=10,
shuffle=True,
class_mode="categorical",
target_size=(156,256))

Found 5386 validated image filenames belonging to 16 classes.


In [0]:
test_datagen=ImageDataGenerator(rescale=1./255.)

In [0]:
 
test_generator=test_datagen.flow_from_dataframe(
dataframe=X_test,
directory=None,
x_col="path",
y_col="label",
subset="training",
batch_size=32,
seed=10,
shuffle=True,
class_mode="categorical",
target_size=(156,256))

Found 15840 validated image filenames belonging to 16 classes.


### Model-1

<pre>
1. Use <a href='https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16'>VGG-16</a> pretrained network without Fully Connected layers and initilize all the weights with Imagenet trained weights. 
2. After VGG-16 network without FC layers, add a new Conv block ( 1 Conv layer and 1 Maxpooling ), 2 FC layers and a output layer to classify 16 classes. You are free to choose any hyperparameters/parameters of conv block, FC layers, output layer. 
3. Final architecture will be <b>INPUT --> VGG-16 without Top layers(FC) --> Conv Layer --> Maxpool Layer --> 2 FC layers --> Output Layer</b>
4. Train only new Conv block, FC layers, output layer. Don't train the VGG-16 network. 

</pre>

In [0]:
from keras import backend as K
K.clear_session()

In [0]:
#input_layer = Input(shape=(156,256,3),name='Input_Layer')
from tensorflow.keras.applications import InceptionV3
#IMG_SHAPE = (156,256,3)

# Create the base model from the pre-trained model MobileNet V2
base_model = InceptionV3(include_top=False,weights='imagenet')

#feature_batch = base_model(image_batch)
#print(feature_batch.shape)
#base_model=keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet') 
base_model.trainable = False





87916544/87910968 [==============================] - 4s 0us/step


In [0]:
from tensorflow.keras import layers


con=Conv2D(32,kernel_size=[3,3],strides=(1,1),padding="valid",activation="relu")(base_model.output)
global_average_layer = layers.GlobalAveragePooling2D()(con)

r1=Dense(1024, activation='relu')(global_average_layer)
r2=Dense(1024, activation='relu')(r1)
output = Dense(16,activation='softmax')(r2)
model = Model(inputs=base_model.input,outputs=output)


In [0]:
%load_ext tensorboard

In [0]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
import datetime
from tensorflow.keras.callbacks import TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

In [0]:
%tensorboard --logdir logs/fit

### Model-2

<pre>
1. Use <a href='https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16'>VGG-16</a> pretrained network without Fully Connected layers and initilize all the weights with Imagenet trained weights.
2. After VGG-16 network without FC layers, don't use FC layers, use conv layers only as Fully connected layer. any FC layer can be converted to a CONV layer. 
This conversion will reduce the No of Trainable parameters in FC layers.
 For example, an FC layer with K=4096 that is looking at some input volume of size 7×7×512 can be equivalently expressed as a CONV layer with F=7,P=0,S=1,K=4096. 
 In other words, we are setting the filter size to be exactly the size of the input volume, and hence the output will simply be 1×1×4096 since only a single depth column “fits” across the input volume, giving identical result as the initial FC layer. 
 You can refer <a href='http://cs231n.github.io/convolutional-networks/#convert'>this</a> link to better understanding of using Conv layer in place of fully connected layers.
3. Final architecture will be VGG-16 without FC layers(without top), 2 Conv layers identical to FC layers, 1 output layer for 16 class classification. <b>INPUT --> VGG-16 without Top layers(FC) --> 2 Conv Layers identical to FC --> Output Layer</b>
3. Train only last 2 Conv layers identical to FC layers, 1 output layer. Don't train the VGG-16 network. 
</pre>

In [0]:
from keras import backend as K
K.clear_session()

In [0]:
from tensorflow.keras.applications import VGG16

In [0]:
for i in range(1):
  model.layers.pop(i)

In [0]:
input_layer = Input(shape=(156,156,3),name='Input_Layer')
from tensorflow.keras.applications import VGG16


# Create the base model from the pre-trained model MobileNet V2
base_model_new = VGG16(include_top=False,weights='imagenet')(input_layer)

#feature_batch = base_model(image_batch)
#print(feature_batch.shape)
#base_model=keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet') 
base_model_new.trainable = False





In [0]:
conv_1=Conv2D(512,kernel_size=[1,1],strides=(1,1),padding="valid",activation="relu")(base_model_new)
conv_2=Conv2D(512,kernel_size=[1,1],strides=(1,1),padding="valid",activation="relu")(conv_1)
output = Dense(16,activation='softmax')(conv_2)
model_new = Model(inputs=input_layer,outputs=output)

In [0]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
from tensorflow.keras.optimizers import Adam
model_new.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
import datetime
from tensorflow.keras.callbacks import TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model_new.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=1
)

### Model-3

<pre>
1. Use same network as Model-2 '<b>INPUT --> VGG-16 without Top layers(FC) --> 2 Conv Layers identical to FC --> Output Layer</b>' and train only Last 6 Layers of VGG-16 network, 2 Conv layers identical to FC layers, 1 output layer.
</pre>

In [0]:
from keras import backend as K
K.clear_session()

In [0]:
from tensorflow.keras.applications import VGG16

In [0]:
input_layer = Input(shape=(156,156,3),name='Input_Layer')
from tensorflow.keras.applications import VGG16
base_model_new3 = VGG16(include_top=False,weights='imagenet')
for layer in base_model_new3.layers[:13]:
    layer.trainable = False
 


In [0]:
for layer in base_model_new.layers:
    print(layer, layer.trainable)

In [0]:
conv_1=Conv2D(512,kernel_size=[1,1],strides=(1,1),padding="valid",activation="relu")(base_model_new3)
conv_2=Conv2D(512,kernel_size=[1,1],strides=(1,1),padding="valid",activation="relu")(conv_1)
output = Dense(16,activation='softmax')(conv_2)
model_new3 = Model(inputs=input_layer,outputs=output)

In [0]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
from tensorflow.keras.optimizers import Adam
model_new3.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
import datetime
from tensorflow.keras.callbacks import TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model_new3.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=1
)